#### 使用HMM实现中文分词任务

在中文分词任务中，观测状态为字，隐藏状态为每个字为BEMS中的一个状态。

- B：词语开始
- E：词语结束
- M：词语中间字
- S：孤立的单个字称为一个词语

In [1]:
import numpy as np
from tqdm import tqdm

In [2]:
#读取数据
with open('./pku_training.utf8', 'r', encoding='UTF-8') as f:
    data = f.readlines()
data = [i.strip() for i in data if len(i.strip()) > 0]  # 注意原始数据中存在空数据，也需要删除
print("length = ",len(data))
data

length =  19054


['迈向  充满  希望  的  新  世纪  ——  一九九八年  新年  讲话  （  附  图片  １  张  ）',
 '中共中央  总书记  、  国家  主席  江  泽民',
 '（  一九九七年  十二月  三十一日  ）',
 '１２月  ３１日  ，  中共中央  总书记  、  国家  主席  江  泽民  发表  １９９８年  新年  讲话  《  迈向  充满  希望  的  新  世纪  》  。  （  新华社  记者  兰  红光  摄  ）',
 '同胞  们  、  朋友  们  、  女士  们  、  先生  们  ：',
 '在  １９９８年  来临  之际  ，  我  十分  高兴  地  通过  中央  人民  广播  电台  、  中国  国际  广播  电台  和  中央  电视台  ，  向  全国  各族  人民  ，  向  香港  特别  行政区  同胞  、  澳门  和  台湾  同胞  、  海外  侨胞  ，  向  世界  各国  的  朋友  们  ，  致以  诚挚  的  问候  和  良好  的  祝愿  ！',
 '１９９７年  ，  是  中国  发展  历史  上  非常  重要  的  很  不  平凡  的  一  年  。  中国  人民  决心  继承  邓  小平  同志  的  遗志  ，  继续  把  建设  有  中国  特色  社会主义  事业  推向  前进  。  中国  政府  顺利  恢复  对  香港  行使  主权  ，  并  按照  “  一国两制  ”  、  “  港人治港  ”  、  高度  自治  的  方针  保持  香港  的  繁荣  稳定  。  中国  共产党  成功  地  召开  了  第十五  次  全国  代表大会  ，  高举  邓小平理论  伟大  旗帜  ，  总结  百年  历史  ，  展望  新  的  世纪  ，  制定  了  中国  跨  世纪  发展  的  行动  纲领  。',
 '在  这  一  年  中  ，  中国  的  改革  开放  和  现代化  建设  继续  向前  迈进  。  国民经济  保持  了  “  高  增长  、  低  通胀  ”  的  良好  发展  态势

In [3]:
I = []  # 存储隐藏状态
O = []  # 存储观测状态
for string in data:
    i = []
    o = []
    string_words = string.split(' ')
    string_words = [i for i in string_words if len(i) > 0]  # 删除产生的空字符串
    for string_word in string_words:
        #print("all: ",string_word,len(string_word))
        if len(string_word) == 1:
           #print("code 1 word:",string_word)
            i.append(3)
            o.append(ord(string_word))  # ord方法将汉字转化为unicode中的数值
        else:
            i.append(0)
            i.extend([2]*(len(string_word) - 2))
            i.append(1)
            o.extend([ord(i) for i in list(string_word)])
    I.append(i)
    O.append(o)

In [4]:
O

[[36808,
  21521,
  20805,
  28385,
  24076,
  26395,
  30340,
  26032,
  19990,
  32426,
  8212,
  8212,
  19968,
  20061,
  20061,
  20843,
  24180,
  26032,
  24180,
  35762,
  35805,
  65288,
  38468,
  22270,
  29255,
  65297,
  24352,
  65289],
 [20013,
  20849,
  20013,
  22830,
  24635,
  20070,
  35760,
  12289,
  22269,
  23478,
  20027,
  24109,
  27743,
  27901,
  27665],
 [65288,
  19968,
  20061,
  20061,
  19971,
  24180,
  21313,
  20108,
  26376,
  19977,
  21313,
  19968,
  26085,
  65289],
 [65297,
  65298,
  26376,
  65299,
  65297,
  26085,
  65292,
  20013,
  20849,
  20013,
  22830,
  24635,
  20070,
  35760,
  12289,
  22269,
  23478,
  20027,
  24109,
  27743,
  27901,
  27665,
  21457,
  34920,
  65297,
  65305,
  65305,
  65304,
  24180,
  26032,
  24180,
  35762,
  35805,
  12298,
  36808,
  21521,
  20805,
  28385,
  24076,
  26395,
  30340,
  26032,
  19990,
  32426,
  12299,
  12290,
  65288,
  26032,
  21326,
  31038,
  35760,
  32773,
  20848,
  32418,


In [5]:
# 计算观测状态O_num和隐藏状态i=I_num
O_num = max(max(O)) + 1
I_num = max(max(I)) + 1

print(I_num, O_num)

# 定义状态转移矩阵
A = np.zeros((I_num, I_num))

# 定义观测矩阵
B = np.zeros((I_num, O_num))

# 定义初始状态
pi = np.zeros(I_num)

4 65342


#### 训练网络即求解参数$\lambda$

一般用EM算法来求解，这里比较简单就直接算了

In [6]:
# 计算初始状态
# 即计算所有可能的输入状态$i={1,2,3,4}$的初始概率
for i in range(I_num):
    for j in range(len(I)):
        pi[i] += (I[j][0] == i)
pi = pi / pi.sum()
pi

array([0.63456492, 0.        , 0.        , 0.36543508])

In [7]:
len(I)

19054

In [8]:
len(I[3])

57

In [9]:
# 计算状态转移矩阵
# 状态转移矩阵的每个元素 a_ij=P(o=v|i=j)，其中v是unicode编码空间中的状态，$j=1,2,3,4$中的一个
for n in tqdm(range(len(I))):  # 遍历数据
    for t in range(len(I[n])-1):  # 遍历时间步
        for i in range(I_num):  # 遍历行数
            for j in range(I_num):  # 遍历列数
                    if I[n][t] == i and I[n][t+1] == j:
                        A[j, i] += 1  # 计数
A /= A.sum(axis=1, keepdims=True) # 归一化
A

100%|██████████| 19054/19054 [00:03<00:00, 5633.91it/s]


array([[0.        , 0.49242151, 0.        , 0.50757849],
       [0.85322422, 0.        , 0.14677578, 0.        ],
       [0.6543287 , 0.        , 0.3456713 , 0.        ],
       [0.        , 0.57792637, 0.        , 0.42207363]])

In [10]:
# 计算观测矩阵
for n in tqdm(range(len(I))[:100]):  # 遍历数据，由于数据量比较大，所以在计算观测矩阵时，仅选择前1000条数据进行训练，要想得到更好的效果可以将训练数据量增大
    for t in range(len(I[n])-1):  # 遍历时间步
        for i in range(I_num):  # 遍历行数
            for j in range(O_num):  # 遍历列数
                if I[n][t] == i and O[n][t] == j:
                    B[i, j] += 1  # 计数
B /= B.sum(axis=1, keepdims=True)
B

  7%|▋         | 7/100 [00:09<02:02,  1.31s/it]


KeyboardInterrupt: 

In [14]:
# 存储中间结果
np.savez('hmm_model_params_epoch1000.npz', A=A, B=B, pi=pi)

#### 使用HMM进行分词

前面我们已经将HMM模型训练完毕，接下来使用训练好的HMM模型，对指定序列进行分词。

这里就可以使用到维特比算法，找到指定观测序列最大概率的隐藏序列

In [30]:
test_text = '我们即将以丰收的喜悦送走牛年，以昂扬的斗志迎来虎年。我们伟大祖国在新的一年，将是充满生机、充满希望的一年。'
#test_text = '通过特征值分解和特征映射，我们证明了如果一个函数如果是正定的'

In [31]:
# 将指定文本转化为观测序列
test_text_ord = [ord(i) for i in list(test_text)]
print(len(test_text_ord))

30


In [39]:
def viterbi(A, B, pi, O):
    result = []
    for t in range(len(O)):
        if t == 0:
            result.append(pi * B[:, O[t]])
        else:
            result.append((result[t-1]@A).max() * B[:, O[t]])
    result = np.stack(result, axis=1)
    cut_result = result.argmax(axis=0)
    return cut_result

In [40]:
# 使用维特比算法求解最大概率的隐藏序列结果
cut_result = viterbi(A, B, pi, test_text_ord)

In [34]:
# 将分词结果还原到句子中，在所有的1和3后面加上|
new_string = ''.join(char + '|' if value in [1, 3] else char for char, value in zip(test_text, cut_result))  
new_string

'通过|特征值分解和特征映射，我们证明了如果一个函数如果是正定的'

In [20]:
def cut(string, A=None, B=None, pi=None, sep=' ', model_params_path=None):
    # 要直接指定HMM的三个参数，要么就指定缓存的npz文件路径，如果都不指定就报错
    if (model_params_path == None) and np.any(A == None):
        raise ValueError('请指定模型参数！')
    if model_params_path:
        model_file = np.load(model_params_path)
        A, B, pi = model_file['A'], model_file['B'], model_file['pi']
    # 将指定文本转化为观测序列
    test_text_ord = [ord(i) for i in list(string)]
    # 使用维特比算法求解最大概率的隐藏序列结果
    cut_result = viterbi(A, B, pi, test_text_ord)
    # 将分词结果还原到句子中，在所有的1和3后面加上空格
    new_string = ''.join(char + sep if value in [1, 3] else char for char, value in zip(string, cut_result))  
    return new_string

In [36]:
cut('台湾是中国领土不可分割的一部分', sep='|', model_params_path='hmm_model_params_epoch1000.npz')

'台湾|是|中国|领土|不|可分割|的|一部分'

In [38]:
cut('通过特征值分解和特征映射是的我真证明的，我们证明了如果一个函数如果是正定的', sep='|', model_params_path='hmm_model_params_epoch1000.npz')

'通过|特征值分解和特征映射是的我真证明的，我们证明了如果一个函数如果是正定的'